#1. Import các thư viện cần thiết cho việc crawl dữ liệu

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re

#2. Tạo HTTP headers dùng để giả lập request từ trình duyệt

Sử dụng để gửi request khi crawl dữ liệu, giúp giả lập trình duyệt thật và hạn chế bị chặn.

In [ ]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
    "Referer": "https://google.com"
}

#3. Xây dựng hàm crawl dữ liệu từ trang web vnexpress.net

In [ ]:
def crawl_vnexpress_data_by_category(category):

    article_metadata = []

    for i in range(20):

        category_url = f"https://vnexpress.net/{category}-p{i+1}"
        try:
            category_response = requests.get(category_url, headers=HEADERS, timeout=10)
            category_response.raise_for_status()   # báo lỗi nếu không phải 200
            category_response.encoding = category_response.apparent_encoding

            print(f"Đang thu thập dữ liệu tại link: {category_url}")

            category_soup = BeautifulSoup(category_response.text, 'html.parser')

            articles = category_soup.find_all("article", class_ = re.compile("^item-new"))
            for article in articles:
                title_tag = article.find("a")
                if title_tag is None:
                    continue
                title = title_tag.get("title")
                url = article.find("a").get("href")
                description_tag = article.find("p", class_ = "description")
                if description_tag is None:
                    continue
                area_tag = description_tag.find("span", class_ = "location-stamp")
                area = area_tag.get_text(strip=True) if area_tag else None
                description_raw = description_tag.get_text(strip=True)
                if area and description_raw.startswith(area):
                    description = description_raw[len(area):].strip()
                else:
                    description = description_raw

                article_metadata.append({
                    "Title": title,
                    "Description": description,
                    "Category": category,
                    "Platfrom": "vnexpress",
                    "Area": area,
                    "URL": url
                    })


        except requests.exceptions.RequestException as e:
            print("Lỗi khi truy cập trang:", e)

    return article_metadata

#4. Xây dựng hàm crawl dữ liệu từ trang web vietnamnet.vn

In [ ]:
def crawl_vietnamnet_data_by_category(category):

    article_metadata = []

    for i in range(30):

        category_url = f"https://vietnamnet.vn/{category}-page{i+1}"
        try:
            category_response = requests.get(category_url, headers=HEADERS, timeout=10)
            category_response.raise_for_status()   # báo lỗi nếu không phải 200
            category_response.encoding = category_response.apparent_encoding

            print(f"Đang thu thập dữ liệu tại link: {category_url}")

            category_soup = BeautifulSoup(category_response.text, 'html.parser')

            articles = category_soup.find_all("div", class_ = re.compile("^horizontalPost sm"))
            for article in articles:
                title = article.find("a").get("title").strip()
                url = article.find("a").get("href")
                description_tag = article.find("div", class_ = "horizontalPost__main-desc").get_text(strip=True)
                parts = description_tag.split(" - ", 1)
                if parts[0].isupper():   # kiểm tra phần đầu in hoa
                    area = parts[0].strip()
                    description = parts[1].strip() if len(parts) > 1 else None
                else:
                    area = None
                    description = description_tag

                article_metadata.append({
                            "Title": title,
                            "Description": description,
                            "Category": mapping.get(category, category),
                            "Platfrom": "vietnamnet",
                            "Area": area,
                            "URL": url
                            })

        except requests.exceptions.RequestException as e:
                    print("Lỗi khi truy cập trang:", e)

    return article_metadata

#5. Crawl dữ liệu bài viết theo danh mục từ VnExpress và VietnamNet


In [ ]:
categories_vnexpress = ["thoi-su", "the-gioi", "khoa-hoc-cong-nghe", "giao-duc", "kinh-doanh", "bat-dong-san", "suc-khoe",
            "the-thao", "giai-tri", "phap-luat", "du-lich"]

categories_vietnamnet = ["thoi-su", "the-gioi", "cong-nghe", "giao-duc", "kinh-doanh", "bat-dong-san", "suc-khoe",
                         "the-thao", "van-hoa-giai-tri", "phap-luat", "du-lich"]

mapping = {
    "cong-nghe": "khoa-hoc-cong-nghe",
    "van-hoa-giai-tri": "giai-tri"
}

metadata = []

for category in categories_vnexpress:
    category_metadata = crawl_vnexpress_data_by_category(category)
    metadata += category_metadata

for category in categories_vietnamnet:
    category_metadata = crawl_vietnamnet_data_by_category(category)
    metadata += category_metadata

Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p1
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p2
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p3
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p4
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p5
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p6
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p7
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p8
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p9
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p10
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p11
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p12
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p13
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p14
Đang thu thập dữ liệu tại link: https://vnexpress.net/thoi-su-p15
Đang thu thập dữ li

In [ ]:
print(len(metadata))

15039


#6. Chuyển metadata thành DataFrame và lưu ra file CSV

In [ ]:
df = pd.DataFrame(metadata)
df.to_csv('articles_dataset_raw.csv', index=True, encoding='utf-8-sig')
df

,Title,Description,Category,Platfrom,Area,URL
0,Cháy lớn quán bar ở Đồng Nai,"Lửa bùng lên ở quán bar, karaoke rộng hơn 300 ...",thoi-su,vnexpress,None,https://vnexpress.net/chay-lon-quan-bar-o-dong...
1,Dùng ngân sách Trung ương xây nhà mới cho ngườ...,Thủ tướng giao Bộ Tài chính đề xuất sử dụng ng...,thoi-su,vnexpress,None,https://vnexpress.net/dung-ngan-sach-trung-uon...
2,6 ôtô tông liên hoàn trên cao tốc Trung Lương,Sáu ôtô va chạm liên hoàn trên cao tốc TP HCM ...,thoi-su,vnexpress,Tây Ninh,https://vnexpress.net/6-oto-tong-lien-hoan-tre...
3,"Quân nhân Việt - Ấn diễn tập phá mìn, cứu hộ đ...","Trải qua ba tình huống thực binh, quân nhân Vi...",thoi-su,vnexpress,Hà Nội,https://vnexpress.net/quan-nhan-viet-an-dien-t...
4,Khởi công mở rộng cao tốc TP HCM - Trung Lương...,Tuyến cao tốc dài gần 100 km nối TP HCM với mi...,thoi-su,vnexpress,None,https://vnexpress.net/khoi-cong-mo-rong-cao-to...
...,...,...,...,...,...,...
15034,"Đang bay trên đỉnh núi, dù lượn của người đàn ...","Dù đã có kinh nghiệm chơi dù lượn lâu năm, son...",du-lich,vietnamnet,TRUNG QUỐC,https://vietnamnet.vn/dang-bay-tren-dinh-nui-d...
15035,Quán xôi 3 đời mẹ truyền con nối ở TPHCM đắt k...,Sau khi lọt vào danh sách Michelin Selected (n...,du-lich,vietnamnet,None,https://vietnamnet.vn/quan-xoi-3-doi-me-truyen...
15036,Bên trong ‘trận địa’ pháo hoa quốc tế Đà Nẵng ...,Trước thềm đêm khai mạc Lễ hội pháo hoa quốc t...,du-lich,vietnamnet,None,https://vietnamnet.vn/ben-trong-tran-dia-phao-...
15037,Lễ hội dâng đăng độc đáo mừng Lễ Vía Bà tại nú...,Từ ngày 30/5 - 1/6/2025 (nhằm ngày 4 - 6/5 âm ...,du-lich,vietnamnet,None,https://vietnamnet.vn/le-hoi-dang-dang-doc-dao...
